In [4]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.cross_validation import cross_val_score
from nltk.tokenize import word_tokenize,sent_tokenize
from sklearn.feature_extraction.text import CountVectorizer
import re

In [5]:
train_data = pd.read_table('Data/training_set_rel3.tsv',encoding = "ISO-8859-1")
train_data_except_set2 = train_data.loc[train_data.essay_set != 2,['essay_set','essay','domain1_score']]
train_data_except_set2.columns = ['essay_set','essay','score']
train_data_set2_1 = train_data.loc[train_data['essay_set'] == 2 ,['essay_set','essay','domain1_score']]
train_data_set2_1['essay_set'] = "2_1"
train_data_set2_1.columns = ['essay_set','essay','score']
train_data_set2_2 = train_data.loc[train_data['essay_set'] == 2 ,['essay_set','essay','domain2_score']]
train_data_set2_2['essay_set'] = "2_2"
train_data_set2_2.columns = ['essay_set','essay','score']
train_data_format = pd.concat([train_data_except_set2,train_data_set2_2,train_data_set2_2])

In [6]:
def get_distinct_word_count(text):
    word_list = word_tokenize(text)
    return len(set(word_list))

def get_word_count_token(text):
    word_list = word_tokenize(text)
    return len(word_list)

def get_sentence_count(text):
    sent_list = sent_tokenize(text)
    return len(sent_list)

def get_character_count(text):
    return len(text)

def get_word_count(text):
    return len(re.findall(r"\s", text))+1

In [7]:
train_data_format['length'] = train_data_format['essay'].apply(get_character_count)
train_data_format['word_count'] = train_data_format['essay'].apply(get_word_count)

In [8]:
train_data_format['distinct_word'] = train_data_format['essay'].apply(get_distinct_word_count)
train_data_format['sentence_count'] = train_data_format['essay'].apply(get_sentence_count)

In [9]:
train_data_format.head(1)

,essay_set,essay,score,length,word_count,distinct_word,sentence_count
0,1,"Dear local newspaper, I think effects computer...",8.0,1875,338,181,16


In [10]:
test_data = pd.read_table('Data/valid_set.tsv',encoding = "ISO-8859-1")
test_data_except_set2 = test_data.loc[test_data.essay_set != 2,['essay_set','essay','domain1_predictionid']]
test_data_except_set2.columns = ['essay_set','essay','predictionid']
test_data_set2_1 = test_data.loc[test_data['essay_set'] == 2 ,['essay_set','essay','domain1_predictionid']]
test_data_set2_1['essay_set'] = "2_1"
test_data_set2_1.columns = ['essay_set','essay','predictionid']
test_data_set2_2 = test_data.loc[test_data['essay_set'] == 2 ,['essay_set','essay','domain2_predictionid']]
test_data_set2_2['essay_set'] = "2_2"
test_data_set2_2.columns = ['essay_set','essay','predictionid']

In [11]:
test_data_format = pd.concat([test_data_except_set2,test_data_set2_1,test_data_set2_2])

In [12]:
test_data_format['word_count'] = test_data_format['essay'].apply(get_word_count)
test_data_format['length'] = test_data_format['essay'].apply(get_character_count)

In [13]:
test_data_format['distinct_word'] = test_data_format['essay'].apply(get_distinct_word_count)
test_data_format['sentence_count'] = test_data_format['essay'].apply(get_sentence_count)

In [16]:
train_data_format.head(1)

,essay_set,essay,score,length,word_count,distinct_word,sentence_count
0,1,"Dear local newspaper, I think effects computer...",8.0,1875,338,181,16


In [18]:
test_data_format.head(1)

,essay_set,essay,predictionid,word_count,length,distinct_word,sentence_count
0,1,"Dear @ORGANIZATION1, @CAPS1 more and more peop...",1788.0,230,1365,135,13


In [61]:
sort_by_id = test_data_format.sort_values(by=['predictionid'], axis=0,inplace=True)

In [62]:
X = train_data_format[['length','word_count']]
y = train_data_format[['score']]
X_test = test_data_format[['length','word_count']]

In [63]:
print X.shape
print y.shape
print X_test.shape

(14776, 2)
(14776, 1)
(4818, 2)


In [64]:
X.head(10)

,length,word_count
0,1875,338
1,2288,419
2,1541,279
3,3165,524
4,2569,465
5,1276,246
6,2808,499
7,2724,482
8,2402,443
9,2632,502


In [65]:
y.head(10)

,score
0,8.0
1,9.0
2,7.0
3,10.0
4,8.0
5,8.0
6,10.0
7,10.0
8,9.0
9,9.0


In [66]:
X_test[0:10]

,length,word_count
0,1365,230
1,1605,310
2,2343,441
3,1981,361
4,2517,453
5,2471,460
6,2435,449
7,3064,555
8,1549,295
9,3036,554


In [67]:
randomForest = RandomForestRegressor(n_estimators=100)
randomForest.fit(X,y)
scores = randomForest.predict(X_test)

/root/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  from ipykernel import kernelapp as app


In [68]:
score_round = [round(i) for i in scores]

In [69]:
score_round[0:10]

[3.0, 7.0, 11.0, 13.0, 8.0, 9.0, 8.0, 5.0, 6.0, 7.0]

In [70]:
predictions= zip(test_data_format['predictionid'],score_round)